# FUNCTION

In [ ]:
import torch
import random
import torchvision
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import torch.utils.data as data
import torch.nn as nn
import tool
%matplotlib inline
import importlib
importlib.reload(tool)

In [71]:
class mnist_data(data.Dataset):
    def __init__(self,x,y):
        self.x=x.float()
        self.y=y.float()
    def __len__(self):
        return len(self.x)
    def __getitem__(self,index):
        if self.y is not None:
            return self.x[index].reshape([-1]),self.y[index]
        else:
            return self.x[index].reshape([-1])


In [97]:
class Auto_Learning(nn.Module):
    def __init__(self,input_a,hidden1_a,hidden2_a,hidden3_a,rand_seed=10):
        torch.manual_seed(rand_seed)
        super(Auto_Learning,self).__init__()
        input_a=input_a*input_a
        hidden1_a*=hidden1_a
        hidden2_a*=hidden2_a
        hidden3_a*=hidden3_a

        self.net=nn.Sequential(
            nn.Linear(input_a,hidden1_a),
            nn.ReLU(),
            nn.Linear(hidden1_a,hidden2_a),
            nn.ReLU(),
            nn.Linear(hidden2_a,hidden3_a),
            nn.ReLU(),
            nn.Linear(hidden3_a,hidden2_a),
            nn.ReLU(),
            nn.Linear(hidden2_a,hidden1_a),
            nn.ReLU(),
            nn.Linear(hidden1_a,input_a),
            nn.ReLU()
        )
        self.lossF=nn.MSELoss()
    def forward(self,input):
        return self.net(input)
    
    def trainF(self,train_data,valid_data,learning_rate=0.1,epoch=100,epoch_per=10,batch_size=128,optim=None):
        lossF=self.lossF
        if optim is None:
            optim=torch.optim.Adam(self.parameters(),lr=learning_rate)
        loader=data.DataLoader(train_data,batch_size=batch_size)
        train_record=[]
        valid_record=[]

        for i in range(epoch):
            train_loss=0.0
            for k in range(epoch_per):
                for x,_ in loader:
                    optim.zero_grad()
                    res=self(x)
                    loss=lossF(res,x)
                    loss.backward()
                    optim.step()
                    train_loss+=loss*len(x)
                    pass
            #compute
            train_loss/=len(train_data)
            train_loss/=epoch_per
            train_record.append(train_loss)
            valid_loss=self.valid(valid_data)
            valid_record.append(valid_loss)
            #print
            print(f'{i}\t{train_loss:.4f}\t{valid_loss:.4f}')
    
        return train_record,valid_record
    def valid(self,valid_data,batch_size=128):
        lossF=self.lossF
        loader=data.DataLoader(valid_data)
        total_loss=0.0
        for x,_ in loader:
            res=self(x)
            loss=lossF(res,x).tolist()
            total_loss+=loss*len(x)
        return total_loss/len(valid_data)




# CODE

## data

In [40]:
mnist=torchvision.datasets.MNIST('./mnist')

In [41]:
train_x,test_x=tool.data_split(mnist.train_data,rand=20)
train_y,test_y=tool.data_split(mnist.train_labels,rand=20)
train_x,valid_x=tool.data_split(train_x,rand=10)
train_y,valid_y=tool.data_split(train_y,rand=10)

In [72]:
train_data=mnist_data(train_x,train_y)
valid_data=mnist_data(valid_x,valid_y)
test_data=mnist_data(test_x,test_y)

In [32]:
train_data[0][0].shape

torch.Size([28, 28])

In [69]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

38400
9600
12000


# AUTO_LEARNING

In [100]:
ALmodel=Auto_Learning(28,20,15,10)

In [101]:
tr,vr=ALmodel.trainF(train_data,valid_data,batch_size=10,epoch_per=1)

0	423049265152.0000	7326.9802
1	7288.4048	7326.9802
2	7288.4048	7326.9802
3	7288.4048	7326.9802
4	7288.4048	7326.9802


KeyboardInterrupt: 

In [96]:
ALmodel.valid(valid_data)

7313.5745828882855